In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorNeuropixelsProjectCache
from pathlib import Path
import gzip
import re

# Import the SWRExplorer class
from SWRExplorer import SWRExplorer


/home/acampbell/miniconda3/envs/allensdk_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Initialize the explorer
explorer = SWRExplorer()

# Find best events across all datasets
best_events = explorer.find_best_events_across_datasets(
    min_sw_power=1.5,
    min_duration=0.08,
    max_duration=0.1,
    min_clcorr=0.8,
    exclude_gamma=True,
    exclude_movement=True,
    max_events_per_probe=10  # Get top 10 events from each probe
)

print("\nTop Events Across All Datasets:")
print(best_events.head())

# Plot the best event
explorer.plot_swr_event(
    events_df=best_events,
    event_idx=0,  # Plot the best event
    channel_pyramidal="1100921954",  # Your channel IDs
    channel_radiatum="1100921954"
)

# If you want to filter by speed for specific sessions:
# First get the session
session_id = best_events.index.get_level_values(0)[0].split('_')[1]  # Extract session ID from index
session = cache.get_ecephys_session(int(session_id))

# Then filter by speed
slow_events = explorer.filter_events_by_allensdk_speed(
    events_df=best_events,
    running_speed_df=session.running_speed,
    max_speed=5.0
)

# Plot a slow event
explorer.plot_swr_event(
    events_df=slow_events,
    event_idx=0,
    channel_pyramidal="1100921954",
    channel_radiatum="1100921954"
)

In [ ]:
# Initialize the explorer
explorer = SWRExplorer()

# List available data
explorer.list_available_data()

# Get statistics about all sessions and probes
stats = explorer.get_session_probe_stats()
print("\nSession and Probe Statistics:")
print(stats)


# Find a specific session and probe with good events
dataset = "allen_visbehave_swr_murphylab2024"
session_id = 1098119201  # Replace with your session ID
probe_id = 1098236048  # Replace with your probe ID

# Find the best events based on quality metrics
best_events = explorer.find_best_events(
    dataset=dataset,
    session_id=str(session_id),
    probe_id=str(probe_id),
    min_sw_power=1.5,      # Minimum sharp wave power
    min_duration=0.08,     # Minimum duration in seconds
    max_duration=0.1,      # Maximum duration in seconds
    min_clcorr=0.2,        # Minimum circular-linear correlation
    exclude_gamma=True,    # Exclude events overlapping with gamma
    exclude_movement=True  # Exclude events overlapping with movement
)

print("\nTop 10 Events by Quality:")
print(best_events.head(10))

cache = VisualBehaviorNeuropixelsProjectCache.from_s3_cache(
    cache_dir="/space/scratch/allen_visbehave_data"
)

session = cache.get_ecephys_session(session_id)
wheel_velocity = session.running_speed["speed"]
session.running_speed
# Filter events using Allen SDK speed data
slow_events = explorer.filter_events_by_allensdk_speed(
    events_df=best_events,
    running_speed_df=session.running_speed,
    max_speed=5.0,
    window=0.5,
    agg='max'  # Default is 'max', but you can change it to 'mean', 'median', or 'min'
)

print("\nEvents After Speed Filtering:")
print(slow_events)
# Get speed for all events
slow_events['mean_speeds'] = explorer.get_allensdk_speed(
    running_speed_df=session.running_speed,
    events_df=slow_events,
    window=0.25,
    agg='mean'
)

slow_events['median_speeds'] = explorer.get_allensdk_speed(
    running_speed_df=session.running_speed,
    events_df=slow_events,
    window=0.25,
    agg='median'
)

print(slow_events)

In [14]:
slow_events.head(10)

,Unnamed: 0,start_time,end_time,duration,max_thresh,mean_zscore,median_zscore,max_zscore,min_zscore,area,...,sw_ripple_plv,sw_ripple_mi,sw_ripple_clcorr,overlaps_with_gamma,gamma_overlap_percent,overlaps_with_movement,movement_overlap_percent,max_speed_in_window,mean_speeds,median_speeds
2,5401,9163.293508,9163.378841,0.085333,3.556363,1.874711,1.188715,5.804641,0.004866,0.161197,...,0.025083,0.056820,0.683171,False,0.0,False,43.750000,0.0,0.0,0.0
6,4854,8815.797508,8815.882841,0.085333,2.120317,1.550964,1.455554,3.842037,0.016750,0.133347,...,0.036336,0.060866,0.605595,False,0.0,False,8.593750,0.0,0.0,0.0
8,5518,9237.668174,9237.756174,0.088000,2.235466,1.368545,0.934246,3.386495,0.028209,0.121292,...,0.048634,0.103679,0.603020,False,0.0,False,66.666667,0.0,0.0,0.0
14,5123,8991.150174,8991.241508,0.091333,3.448127,2.301426,2.204115,4.826338,0.075669,0.211668,...,0.014990,0.023524,0.559846,False,0.0,False,0.000000,0.0,0.0,0.0
18,5339,9120.080841,9120.170174,0.089333,2.727139,1.607633,1.088901,4.837445,0.007438,0.144638,...,0.035521,0.093955,0.554708,False,0.0,False,0.000000,0.0,0.0,0.0
19,5211,9045.959508,9046.051508,0.092000,2.348759,1.754833,1.926854,3.815078,0.001861,0.162603,...,0.023253,0.021771,0.536943,False,0.0,False,16.666667,0.0,0.0,0.0
30,6024,9666.105508,9666.195508,0.090000,3.468678,1.967237,1.277949,7.814041,0.004022,0.178359,...,0.031248,0.074551,0.506659,False,0.0,False,0.000000,0.0,0.0,0.0
32,4873,8827.804174,8827.896841,0.092667,2.300332,1.855723,1.830895,3.877086,0.044201,0.173129,...,0.004573,0.085990,0.501744,False,0.0,False,0.000000,0.0,0.0,0.0
33,5223,9053.855508,9053.944174,0.088667,2.157225,1.540338,1.402409,4.058467,0.016332,0.137577,...,0.029617,0.038772,0.500174,False,0.0,False,18.045113,0.0,0.0,0.0
37,5591,9298.743508,9298.832174,0.088667,3.769776,2.110368,1.933172,5.113214,0.010825,0.188462,...,0.027250,0.060400,0.488987,False,0.0,False,0.000000,0.0,0.0,0.0


In [ ]:
explorer.plot_swr_event(dataset, session_id, probe_id, event_idx, 
                      channel_pyramidal=None, channel_radiatum=None)